# Question 1: {20 points}
• Using Postman, create a collection with 4 APIs that refers to the dataset loaded onto S3
as part of assignment 1 or any other dataset

Please reference the following files for complete and functioning code responding to this question. Collection can be loaded into postman for review but I will be shutting down the services initiated for this activity:
* AWS Operations FHebal.postman_collection.json
* aws_restapi.py

# Question 2: {40 points}
• Upload the dataset from S3 into Elasticsearch and provide 4 queries. You are welcome
to use other dataset as well.

In [1]:
from elasticsearch import Elasticsearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3
import json

host = 'search-big-data-cdc-5p7culbjoeubnqjlig6732uhh4.us-east-2.es.amazonaws.com'
host = 'search-big-data-cdc-3hj3bglskm3oyw7oh63tiiseyu.us-east-2.es.amazonaws.com'
region = 'us-east-2'
service = 'es'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service)

es = Elasticsearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = awsauth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

print("COUNT\n")
print(es.count())

print("\nKEYS\n")
print(es.indices.get_mapping().keys())

print("\nBRFSS INDICES\n")
print(json.dumps(es.indices.get_mapping()['brfss'],indent=4))

COUNT



ConnectionError: ConnectionError(HTTPSConnectionPool(host='search-big-data-cdc-3hj3bglskm3oyw7oh63tiiseyu.us-east-2.es.amazonaws.com', port=443): Max retries exceeded with url: /_count (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f7314de0fd0>: Failed to establish a new connection: [Errno -2] Name or service not known',))) caused by: ConnectionError(HTTPSConnectionPool(host='search-big-data-cdc-3hj3bglskm3oyw7oh63tiiseyu.us-east-2.es.amazonaws.com', port=443): Max retries exceeded with url: /_count (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f7314de0fd0>: Failed to establish a new connection: [Errno -2] Name or service not known',)))

In [87]:
# Query #1 Match operator finding anyone with activity of 20 minutes per week

res = es.search(index="brfss",body={
    "query": {
        "match": {
            "Activity_1_weekly_duration_minutes": "20.0"
        }
    }
})
print(json.dumps(res['hits']['hits'],indent=4))

[
    {
        "_index": "brfss",
        "_type": "respondent",
        "_id": "7899",
        "_score": 1.0,
        "_source": {
            "State": "Alaska",
            "File_Month": 3,
            "Interview_Date": "2013-03-06T00:00:00",
            "Interview_Month": 3,
            "Interview_Day": 6,
            "Interview_Year": "2013",
            "SEQN": "2013000601",
            "Primary_Sampling_Unit": "2013000601",
            "Private_Residence": "1",
            "College_Housing": " ",
            "Is_Adult": " ",
            "General_Health": "Very Good",
            "Physical_Health": 3,
            "Mental_Health": 88,
            "Days_of_Poor_Health_past_30_days": 88.0,
            "Sleep_per_day_hours": 7,
            "High_BP": "1",
            "BP_Meds": "2",
            "Cholesterol_Awareness": "1",
            "Last_Cholesterol_Check": "1",
            "High_Cholesterol": "1",
            "Had_Heart_Attack": "2",
            "Had_Heart_Disease": "2",
       

In [88]:
# Query #2 Boolean operater enforcing dictionary containing matches.

res= es.search(index='brfss',body={
        'query':{
            'bool':{
                'must':[{
                        'match':{
                            'Activity_1_weekly_duration_minutes':'20.0'
                        }
                    }]
            }
        }
    })
print(json.dumps(res['hits']['hits'],indent=4))

[
    {
        "_index": "brfss",
        "_type": "respondent",
        "_id": "7899",
        "_score": 1.0,
        "_source": {
            "State": "Alaska",
            "File_Month": 3,
            "Interview_Date": "2013-03-06T00:00:00",
            "Interview_Month": 3,
            "Interview_Day": 6,
            "Interview_Year": "2013",
            "SEQN": "2013000601",
            "Primary_Sampling_Unit": "2013000601",
            "Private_Residence": "1",
            "College_Housing": " ",
            "Is_Adult": " ",
            "General_Health": "Very Good",
            "Physical_Health": 3,
            "Mental_Health": 88,
            "Days_of_Poor_Health_past_30_days": 88.0,
            "Sleep_per_day_hours": 7,
            "High_BP": "1",
            "BP_Meds": "2",
            "Cholesterol_Awareness": "1",
            "Last_Cholesterol_Check": "1",
            "High_Cholesterol": "1",
            "Had_Heart_Attack": "2",
            "Had_Heart_Disease": "2",
       

In [89]:
# Query #3 Filter 
# Find all in Very Good "General_Health", but we want only those who are not very active (peculiar population)
res= es.search(index='brfss',body={
        'query':{
            'bool':{
                'must':{
                    'match':{
                        'General_Health':'Very Good'
                    }
                },
                "filter":{
                    "range":{
                        "Activity_1_weekly_duration_minutes":{
                            "lt":'10.0'
                        }
                    }
                }
            }
        }
    })
print(json.dumps(res['hits']['hits'],indent=4))

[
    {
        "_index": "brfss",
        "_type": "respondent",
        "_id": "754",
        "_score": 1.1381837,
        "_source": {
            "State": "Alabama",
            "File_Month": 9,
            "Interview_Date": "2013-09-24T00:00:00",
            "Interview_Month": 9,
            "Interview_Day": 24,
            "Interview_Year": "2013",
            "SEQN": "2013003196",
            "Primary_Sampling_Unit": "2013003196",
            "Private_Residence": "1",
            "College_Housing": " ",
            "Is_Adult": " ",
            "General_Health": "Very Good",
            "Physical_Health": 88,
            "Mental_Health": 88,
            "Days_of_Poor_Health_past_30_days": "",
            "Sleep_per_day_hours": 7,
            "High_BP": "1",
            "BP_Meds": "1",
            "Cholesterol_Awareness": "1",
            "Last_Cholesterol_Check": "1",
            "High_Cholesterol": "1",
            "Had_Heart_Attack": "2",
            "Had_Heart_Disease": "2",
 

In [128]:
# Query #4 Fuzzy match for inactive people

res= es.search(index='brfss',body={
        'query':{
            'fuzzy':{
                "Activity_category":"Inactive"
            }
        }
    })

print(json.dumps(res['hits']['hits'],indent=4))

[
    {
        "_index": "brfss",
        "_type": "respondent",
        "_id": "76061",
        "_score": 1.327151,
        "_source": {
            "State": "Florida",
            "File_Month": 8,
            "Interview_Date": "2013-08-08T00:00:00",
            "Interview_Month": 8,
            "Interview_Day": 8,
            "Interview_Year": "2013",
            "SEQN": "2013014799",
            "Primary_Sampling_Unit": "2013014799",
            "Private_Residence": "1",
            "College_Housing": " ",
            "Is_Adult": " ",
            "General_Health": "Very Good",
            "Physical_Health": 3,
            "Mental_Health": 2,
            "Days_of_Poor_Health_past_30_days": 88.0,
            "Sleep_per_day_hours": 6,
            "High_BP": "3",
            "BP_Meds": " ",
            "Cholesterol_Awareness": "1",
            "Last_Cholesterol_Check": "1",
            "High_Cholesterol": "2",
            "Had_Heart_Attack": "2",
            "Had_Heart_Disease": "2",
 

# Question 3: {40 points}
• Provide 4 Cypher queries/insights based on the panama dataset loaded as part of
Exercise 4.

# I apologize but I will be turning in Question #3 late. I have been unable to launch a stable EC2 instance after installation of neo4j and jdk.